In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import cv2
from sklearn.model_selection import train_test_split
import os
import pickle
from PIL import Image
from resizeimage import resizeimage

In [2]:
device = torch.device('cpu')  # sad reax only

In [3]:
x = []  # image data, start as list
y = []  # output class data, start as list

for folder in os.listdir("./pictures/"):
    for file in os.listdir("./pictures/" + folder + "/"):
        im = cv2.imread("./pictures/" + folder + "/" + file)
        im = np.reshape(im, (3, 256, 144))
        x.append(im)
        out = np.zeros(9)
        out[int(folder)] = 1
        # out = int(folder)
        y.append(out)

x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.002, random_state=0)

In [4]:
class TrainDataset(Dataset):
    def __init__(self):
        self.len = x_train.shape[0]
        self.x_data = torch.from_numpy(x_train).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_train).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [5]:
class TestDataset(Dataset):
    def __init__(self):
        self.len = x_test.shape[0]
        self.x_data = torch.from_numpy(x_test).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_test).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes=9):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5, 1)
        self.conv2 = nn.Conv2d(5, 10, 5, 1)
        self.fc1 = nn.Linear(20130, 10)
        self.fc2 = nn.Linear(10, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 20130)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
learning_rate = 0.002
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data)
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data))
    if epoch == 10:
        pkl_filename = "pickle_model.pkl"
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)

In [ ]:
if __name__ == '__main__':
    train_dataset = TrainDataset()
    train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

    test_dataset = TestDataset()
    test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

    """for epoch in range(5):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            print(epoch, i, "inputs", inputs.data, "labels", labels.data)"""

    for epoch in range(1, 11):
        train(epoch, train_loader)

        # test(test_loader)

Train Epoch: 1 [0/8089 (0%)]	Loss: 21.957638
Train Epoch: 1 [10/8089 (0%)]	Loss: 1.941788
Train Epoch: 1 [20/8089 (0%)]	Loss: 2.138075
Train Epoch: 1 [30/8089 (0%)]	Loss: 2.112164
Train Epoch: 1 [40/8089 (0%)]	Loss: 2.107529
Train Epoch: 1 [50/8089 (1%)]	Loss: 2.249937
Train Epoch: 1 [60/8089 (1%)]	Loss: 1.929108
Train Epoch: 1 [70/8089 (1%)]	Loss: 2.130854
Train Epoch: 1 [80/8089 (1%)]	Loss: 2.134640
Train Epoch: 1 [90/8089 (1%)]	Loss: 2.109948
Train Epoch: 1 [100/8089 (1%)]	Loss: 2.095120
Train Epoch: 1 [110/8089 (1%)]	Loss: 2.251592
Train Epoch: 1 [120/8089 (1%)]	Loss: 2.090375
Train Epoch: 1 [130/8089 (2%)]	Loss: 2.143380
Train Epoch: 1 [140/8089 (2%)]	Loss: 2.504224
Train Epoch: 1 [150/8089 (2%)]	Loss: 1.926766
Train Epoch: 1 [160/8089 (2%)]	Loss: 2.228300
Train Epoch: 1 [170/8089 (2%)]	Loss: 2.235242
Train Epoch: 1 [180/8089 (2%)]	Loss: 2.129154
Train Epoch: 1 [190/8089 (2%)]	Loss: 2.527247
Train Epoch: 1 [200/8089 (2%)]	Loss: 2.088508
Train Epoch: 1 [210/8089 (3%)]	Loss: 1.95410

Train Epoch: 1 [1740/8089 (22%)]	Loss: 2.233582
Train Epoch: 1 [1750/8089 (22%)]	Loss: 2.230055
Train Epoch: 1 [1760/8089 (22%)]	Loss: 2.109023
Train Epoch: 1 [1770/8089 (22%)]	Loss: 2.145427
Train Epoch: 1 [1780/8089 (22%)]	Loss: 2.072958
Train Epoch: 1 [1790/8089 (22%)]	Loss: 2.094811
Train Epoch: 1 [1800/8089 (22%)]	Loss: 2.144136
Train Epoch: 1 [1810/8089 (22%)]	Loss: 2.164985
Train Epoch: 1 [1820/8089 (22%)]	Loss: 2.161087
Train Epoch: 1 [1830/8089 (23%)]	Loss: 2.093873
Train Epoch: 1 [1840/8089 (23%)]	Loss: 2.248968
Train Epoch: 1 [1850/8089 (23%)]	Loss: 2.057988
Train Epoch: 1 [1860/8089 (23%)]	Loss: 2.158326
Train Epoch: 1 [1870/8089 (23%)]	Loss: 2.312831
Train Epoch: 1 [1880/8089 (23%)]	Loss: 2.164314
Train Epoch: 1 [1890/8089 (23%)]	Loss: 2.073621
Train Epoch: 1 [1900/8089 (23%)]	Loss: 2.073682
Train Epoch: 1 [1910/8089 (24%)]	Loss: 2.208257
Train Epoch: 1 [1920/8089 (24%)]	Loss: 2.170280
Train Epoch: 1 [1930/8089 (24%)]	Loss: 2.066548
Train Epoch: 1 [1940/8089 (24%)]	Loss: 2

Train Epoch: 1 [3440/8089 (43%)]	Loss: 2.237362
Train Epoch: 1 [3450/8089 (43%)]	Loss: 2.117720
Train Epoch: 1 [3460/8089 (43%)]	Loss: 2.115983
Train Epoch: 1 [3470/8089 (43%)]	Loss: 2.231644
Train Epoch: 1 [3480/8089 (43%)]	Loss: 2.032096
Train Epoch: 1 [3490/8089 (43%)]	Loss: 2.134650
Train Epoch: 1 [3500/8089 (43%)]	Loss: 2.232466
Train Epoch: 1 [3510/8089 (43%)]	Loss: 2.090316
Train Epoch: 1 [3520/8089 (44%)]	Loss: 2.173427
Train Epoch: 1 [3530/8089 (44%)]	Loss: 2.171220
Train Epoch: 1 [3540/8089 (44%)]	Loss: 2.230047
Train Epoch: 1 [3550/8089 (44%)]	Loss: 2.107523
Train Epoch: 1 [3560/8089 (44%)]	Loss: 2.121509
Train Epoch: 1 [3570/8089 (44%)]	Loss: 2.233105
Train Epoch: 1 [3580/8089 (44%)]	Loss: 2.109213
Train Epoch: 1 [3590/8089 (44%)]	Loss: 2.259682
Train Epoch: 1 [3600/8089 (45%)]	Loss: 2.227678
Train Epoch: 1 [3610/8089 (45%)]	Loss: 2.028458
Train Epoch: 1 [3620/8089 (45%)]	Loss: 2.029547
Train Epoch: 1 [3630/8089 (45%)]	Loss: 2.234216
Train Epoch: 1 [3640/8089 (45%)]	Loss: 2

Train Epoch: 1 [5140/8089 (64%)]	Loss: 2.052222
Train Epoch: 1 [5150/8089 (64%)]	Loss: 2.142500
Train Epoch: 1 [5160/8089 (64%)]	Loss: 2.152564
Train Epoch: 1 [5170/8089 (64%)]	Loss: 2.070611
Train Epoch: 1 [5180/8089 (64%)]	Loss: 2.142779
Train Epoch: 1 [5190/8089 (64%)]	Loss: 2.159231
Train Epoch: 1 [5200/8089 (64%)]	Loss: 2.240547
Train Epoch: 1 [5210/8089 (64%)]	Loss: 2.281581
Train Epoch: 1 [5220/8089 (65%)]	Loss: 2.131053
Train Epoch: 1 [5230/8089 (65%)]	Loss: 2.234822
Train Epoch: 1 [5240/8089 (65%)]	Loss: 2.230475
Train Epoch: 1 [5250/8089 (65%)]	Loss: 2.178438
Train Epoch: 1 [5260/8089 (65%)]	Loss: 2.028092
Train Epoch: 1 [5270/8089 (65%)]	Loss: 2.100784
Train Epoch: 1 [5280/8089 (65%)]	Loss: 2.219857
Train Epoch: 1 [5290/8089 (65%)]	Loss: 2.271048
Train Epoch: 1 [5300/8089 (66%)]	Loss: 2.079484
Train Epoch: 1 [5310/8089 (66%)]	Loss: 2.089058
Train Epoch: 1 [5320/8089 (66%)]	Loss: 2.264865
Train Epoch: 1 [5330/8089 (66%)]	Loss: 2.079914
Train Epoch: 1 [5340/8089 (66%)]	Loss: 2

Train Epoch: 1 [6840/8089 (85%)]	Loss: 2.231279
Train Epoch: 1 [6850/8089 (85%)]	Loss: 2.229363
Train Epoch: 1 [6860/8089 (85%)]	Loss: 2.234565
Train Epoch: 1 [6870/8089 (85%)]	Loss: 2.700637
Train Epoch: 1 [6880/8089 (85%)]	Loss: 2.118609
Train Epoch: 1 [6890/8089 (85%)]	Loss: 2.041010
Train Epoch: 1 [6900/8089 (85%)]	Loss: 2.246296
Train Epoch: 1 [6910/8089 (85%)]	Loss: 2.062601
Train Epoch: 1 [6920/8089 (86%)]	Loss: 2.154249
Train Epoch: 1 [6930/8089 (86%)]	Loss: 2.255588
Train Epoch: 1 [6940/8089 (86%)]	Loss: 2.213477
Train Epoch: 1 [6950/8089 (86%)]	Loss: 2.254031
Train Epoch: 1 [6960/8089 (86%)]	Loss: 2.114367
Train Epoch: 1 [6970/8089 (86%)]	Loss: 2.112323
Train Epoch: 1 [6980/8089 (86%)]	Loss: 2.214297
Train Epoch: 1 [6990/8089 (86%)]	Loss: 2.290705
Train Epoch: 1 [7000/8089 (87%)]	Loss: 2.050087
Train Epoch: 1 [7010/8089 (87%)]	Loss: 2.049561
Train Epoch: 1 [7020/8089 (87%)]	Loss: 2.686555
Train Epoch: 1 [7030/8089 (87%)]	Loss: 2.203376
Train Epoch: 1 [7040/8089 (87%)]	Loss: 2

...

Train Epoch: 9 [210/8089 (3%)]	Loss: 2.249123
Train Epoch: 9 [220/8089 (3%)]	Loss: 2.136539
Train Epoch: 9 [230/8089 (3%)]	Loss: 2.050147
Train Epoch: 9 [240/8089 (3%)]	Loss: 2.050638
Train Epoch: 9 [250/8089 (3%)]	Loss: 2.230257
Train Epoch: 9 [260/8089 (3%)]	Loss: 2.180688
Train Epoch: 9 [270/8089 (3%)]	Loss: 2.086909
Train Epoch: 9 [280/8089 (3%)]	Loss: 2.176472
Train Epoch: 9 [290/8089 (4%)]	Loss: 2.175747
Train Epoch: 9 [300/8089 (4%)]	Loss: 2.236941
Train Epoch: 9 [310/8089 (4%)]	Loss: 2.022489
Train Epoch: 9 [320/8089 (4%)]	Loss: 2.187450
Train Epoch: 9 [330/8089 (4%)]	Loss: 2.049762
Train Epoch: 9 [340/8089 (4%)]	Loss: 2.276771
Train Epoch: 9 [350/8089 (4%)]	Loss: 2.270503
Train Epoch: 9 [360/8089 (4%)]	Loss: 2.232586
Train Epoch: 9 [370/8089 (5%)]	Loss: 2.291690
Train Epoch: 9 [380/8089 (5%)]	Loss: 2.006341
Train Epoch: 9 [390/8089 (5%)]	Loss: 2.250154
Train Epoch: 9 [400/8089 (5%)]	Loss: 2.093287
Train Epoch: 9 [410/8089 (5%)]	Loss: 2.090526
Train Epoch: 9 [420/8089 (5%)]	Los

Train Epoch: 9 [1940/8089 (24%)]	Loss: 2.023803
Train Epoch: 9 [1950/8089 (24%)]	Loss: 2.319012
Train Epoch: 9 [1960/8089 (24%)]	Loss: 2.044730
Train Epoch: 9 [1970/8089 (24%)]	Loss: 2.296902
Train Epoch: 9 [1980/8089 (24%)]	Loss: 2.241439
Train Epoch: 9 [1990/8089 (25%)]	Loss: 2.031701
Train Epoch: 9 [2000/8089 (25%)]	Loss: 2.012091
Train Epoch: 9 [2010/8089 (25%)]	Loss: 2.024786
Train Epoch: 9 [2020/8089 (25%)]	Loss: 2.021278
Train Epoch: 9 [2030/8089 (25%)]	Loss: 2.019872
Train Epoch: 9 [2040/8089 (25%)]	Loss: 2.303098
Train Epoch: 9 [2050/8089 (25%)]	Loss: 2.001851
Train Epoch: 9 [2060/8089 (25%)]	Loss: 2.649575
Train Epoch: 9 [2070/8089 (26%)]	Loss: 2.160155
Train Epoch: 9 [2080/8089 (26%)]	Loss: 2.295831
Train Epoch: 9 [2090/8089 (26%)]	Loss: 2.127315
Train Epoch: 9 [2100/8089 (26%)]	Loss: 2.296243
Train Epoch: 9 [2110/8089 (26%)]	Loss: 2.135569
Train Epoch: 9 [2120/8089 (26%)]	Loss: 2.045801
Train Epoch: 9 [2130/8089 (26%)]	Loss: 2.300204
Train Epoch: 9 [2140/8089 (26%)]	Loss: 2

Train Epoch: 9 [3640/8089 (45%)]	Loss: 2.101947
Train Epoch: 9 [3650/8089 (45%)]	Loss: 2.100690
Train Epoch: 9 [3660/8089 (45%)]	Loss: 2.081881
Train Epoch: 9 [3670/8089 (45%)]	Loss: 2.071236
Train Epoch: 9 [3680/8089 (45%)]	Loss: 2.261097
Train Epoch: 9 [3690/8089 (46%)]	Loss: 2.688845
Train Epoch: 9 [3700/8089 (46%)]	Loss: 2.065733
Train Epoch: 9 [3710/8089 (46%)]	Loss: 2.116773
Train Epoch: 9 [3720/8089 (46%)]	Loss: 2.063925
Train Epoch: 9 [3730/8089 (46%)]	Loss: 2.268008
Train Epoch: 9 [3740/8089 (46%)]	Loss: 2.133029
Train Epoch: 9 [3750/8089 (46%)]	Loss: 2.136184
Train Epoch: 9 [3760/8089 (46%)]	Loss: 2.048146
Train Epoch: 9 [3770/8089 (47%)]	Loss: 2.193329
Train Epoch: 9 [3780/8089 (47%)]	Loss: 2.046779
Train Epoch: 9 [3790/8089 (47%)]	Loss: 2.081404
Train Epoch: 9 [3800/8089 (47%)]	Loss: 2.185605
Train Epoch: 9 [3810/8089 (47%)]	Loss: 2.152652
Train Epoch: 9 [3820/8089 (47%)]	Loss: 2.099612
Train Epoch: 9 [3830/8089 (47%)]	Loss: 2.079902
Train Epoch: 9 [3840/8089 (47%)]	Loss: 2

Train Epoch: 9 [5340/8089 (66%)]	Loss: 2.043266
Train Epoch: 9 [5350/8089 (66%)]	Loss: 2.046278
Train Epoch: 9 [5360/8089 (66%)]	Loss: 2.048497
Train Epoch: 9 [5370/8089 (66%)]	Loss: 2.211248
Train Epoch: 9 [5380/8089 (67%)]	Loss: 2.206971
Train Epoch: 9 [5390/8089 (67%)]	Loss: 2.688872
Train Epoch: 9 [5400/8089 (67%)]	Loss: 2.129750
Train Epoch: 9 [5410/8089 (67%)]	Loss: 2.256174
Train Epoch: 9 [5420/8089 (67%)]	Loss: 2.254034
Train Epoch: 9 [5430/8089 (67%)]	Loss: 2.203109
Train Epoch: 9 [5440/8089 (67%)]	Loss: 2.144196
Train Epoch: 9 [5450/8089 (67%)]	Loss: 2.184322
Train Epoch: 9 [5460/8089 (67%)]	Loss: 2.180620
Train Epoch: 9 [5470/8089 (68%)]	Loss: 2.140487
Train Epoch: 9 [5480/8089 (68%)]	Loss: 2.138504
Train Epoch: 9 [5490/8089 (68%)]	Loss: 2.247852
Train Epoch: 9 [5500/8089 (68%)]	Loss: 2.111785
Train Epoch: 9 [5510/8089 (68%)]	Loss: 2.247836
Train Epoch: 9 [5520/8089 (68%)]	Loss: 2.041024
Train Epoch: 9 [5530/8089 (68%)]	Loss: 2.182628
Train Epoch: 9 [5540/8089 (68%)]	Loss: 2

Train Epoch: 9 [7040/8089 (87%)]	Loss: 2.778035
Train Epoch: 9 [7050/8089 (87%)]	Loss: 2.188764
Train Epoch: 9 [7060/8089 (87%)]	Loss: 1.981099
Train Epoch: 9 [7070/8089 (87%)]	Loss: 2.125881
Train Epoch: 9 [7080/8089 (88%)]	Loss: 1.977046
Train Epoch: 9 [7090/8089 (88%)]	Loss: 2.204931
Train Epoch: 9 [7100/8089 (88%)]	Loss: 2.156136
Train Epoch: 9 [7110/8089 (88%)]	Loss: 2.054933
Train Epoch: 9 [7120/8089 (88%)]	Loss: 1.989237
Train Epoch: 9 [7130/8089 (88%)]	Loss: 2.155509
Train Epoch: 9 [7140/8089 (88%)]	Loss: 2.186053
Train Epoch: 9 [7150/8089 (88%)]	Loss: 2.275928
Train Epoch: 9 [7160/8089 (89%)]	Loss: 2.042807
Train Epoch: 9 [7170/8089 (89%)]	Loss: 2.148272
Train Epoch: 9 [7180/8089 (89%)]	Loss: 2.213654
Train Epoch: 9 [7190/8089 (89%)]	Loss: 2.255147
Train Epoch: 9 [7200/8089 (89%)]	Loss: 2.169503
Train Epoch: 9 [7210/8089 (89%)]	Loss: 2.022479
Train Epoch: 9 [7220/8089 (89%)]	Loss: 2.017409
Train Epoch: 9 [7230/8089 (89%)]	Loss: 2.198929
Train Epoch: 9 [7240/8089 (90%)]	Loss: 2

In [ ]:
feedback = ["Move down and right", "Move down", "Move down and left", "Move right", "Perfect!", "Move left", "Move up and right", "Move up", "Move right"]
while True:
    for file in os.listdir("../../../../Downloads/"):
        l = []
        if 'ss_file' in file:
            try:
                with open("../../../../Downloads/" + file, 'r+b') as f:
                    with Image.open(f) as image:
                        resized_image = resizeimage.resize_cover(image, [256, 144])
                        resized_image.save("../../../../Downloads/" + file, image.format)
                im = cv2.imread("../../../../Downloads/" + file)
                im = np.reshape(im, (1, 3, 256, 144))
                x = np.array(im)
                x = torch.from_numpy(x).type(torch.FloatTensor)
                output = model(x)
                try:
                    os.remove("../../../../../natha/thingy/htdocs/hophacks/file.txt")
                except:
                    pass
                output = output.data.numpy().tolist()
                output = output[0]
                output = output.index(max(output))
                %store feedback[output] >> ../../../../../natha/thingy/htdocs/hophacks/file.txt
                try:
                    os.remove("../../../../Downloads/" + file)
                except:
                    pass
            except:
                pass